In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from qvix import get_qvix, update_qvix, get_hist_vol
from option_backtest import FixedHedge, DynamicHedge

In [ ]:
# 从期权论坛更新qvix数据
qvix = update_qvix()

In [ ]:
# 获取qvix数据和HV数据
qvix_df = get_qvix()
hist_vol = get_hist_vol()
hist_vol['HV_20_40'] = (hist_vol['HV_20'] + hist_vol['HV_40']) / 2

hist_vol['date'] = hist_vol['date'].map(lambda d_str: datetime.strptime(d_str, '%Y-%m-%d'))
hist_vol.set_index('date', inplace=True)
hist_vol.head()

In [ ]:
# 绘制hv走势图
disp_list = [qvix_df['close'], hist_vol['HV_20_40']]
df_combine = pd.concat(disp_list, axis=1).dropna()
df_combine.head()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_combine.index, y=df_combine['HV_20_40'], mode='lines+markers', name='HV'))
fig.add_trace(go.Scatter(x=df_combine.index, y=df_combine['close'], mode='lines+markers', name='IV'))
fig.show()

In [ ]:
# 绘制qvix和hv盒须图以及散点分布
fig = go.Figure()
fig.add_trace(go.Box(y=qvix_df['close'], name='qvix', boxpoints='all'))
fig.add_trace(go.Box(y=hist_vol['HV_20_40'], name='hv_20&40', boxpoints='all'))
# fig.update_traces(boxpoints='all', jitter=0)
fig.show()

In [ ]:
# 绘制期权论坛qvix盒须图
fig = px.box(qvix_df, y='high', points='all')
fig.show()


In [ ]:
# IV20&40盒须图
fig = px.box(hist_vol, y='HV_20_40', points='all')
fig.show()

In [ ]:
# 绘制历史波动率锥
hist_vol = get_hist_vol()
columns = [name for name in hist_vol.columns if name.startswith('HV')]
quantile_df = hist_vol[columns].quantile(np.arange(.1, 1.1, .1))
quantile_df.index = list(range(10, 110, 10))

fig = go.Figure()
for index, name in quantile_df.iterrows():
    fig.add_trace(go.Scatter(x=columns, y=quantile_df.loc[index].values, mode='lines+markers', name=str(index)))

fig.show()